# Model Comparisons in Demand Forecasting for E-Commerce

**Objective:** an ecommerce platform aimed to predict customer demand to optimize pricing and inventory management.

**Approach:** time series data of customer browsing behavior, product views, and historical sales were collected. Multiple models were used to capture seasonality and long-term dependencies.

**Outcome:** accurate demand forecasts helped the e-commerce platform optimize pricing, improve product recommendations, and efficiently manage inventory, resulting in increased sales and customer retention.

In [48]:
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# Load the data


## Model Comparison Results


|Model|RMSE|
|---|---|
|Holt's Winter Exponential Smoothing|9283|
